In [1]:
from keras.layers import Conv2D,Flatten,Dense,MaxPooling2D
from os import listdir
from keras.utils import img_to_array
import cv2
from sklearn.model_selection import train_test_split
import numpy as np

# set random seed
np.random.seed(42)

# root folder
root_dir="data/brain-tumor-datasets/"

In [2]:
def convert_img_to_tensor(fpath):
    #read image
    img = cv2.imread(fpath)
        img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    img = cv2.resize(img,(256,256))
    
    #converts image to array
    res = img_to_array(img)
    
    return res

def get_img_data_and_label(root_dir):
    dire=listdir(root_dir)
    
    image_dataset=[]
    image_label=[]
    classes=[]
    
    binary_label=[]
    i=0
    
    for subdir in dire:
        binary_label.append(i)
        classes.append(subdir)
        i+=1
        
    index=0
    
    for subdir in dire:
        skin_img_list=listdir(f"{root_dir}/{subdir}")
        
        for imgfile in skin_img_list:
            
            filepath=f"{root_dir}/{subdir}/{imgfile}"
            #convert image to array
            res=convert_img_to_tensor(filepath)
            #add data to dataset list
            image_dataset.append(res)
            image_label.append(binary_label[index])
            
        index+=1
        
    return image_dataset,image_label,len(binary_label),classes

In [3]:

image_dataset,image_labels,NoOfOutputLayer,classes = get_img_data_and_label(root_dir)

In [4]:
print(NoOfOutputLayer)
print(len(image_labels))
print(len(image_dataset))
print(image_dataset[0].shape)
print(classes)

2
3000
3000
(256, 256, 3)
['no', 'yes']


In [11]:
xtrain,xtest,ytrain,ytest=train_test_split(image_dataset,image_labels,test_size=0.2,random_state=100)

from keras.utils import to_categorical
ytrain=to_categorical(ytrain)
ytest=to_categorical(ytest)



xtrain = np.array(xtrain, dtype=np.float16)/ 255.0
xtrain = xtrain.reshape(-1,256,256,3)
xtest = np.array(xtest, dtype=np.float16)/ 255.0
xtest = xtest.reshape(-1,256,256,3)


print(xtrain.shape)
print(ytrain.shape)
print(xtest.shape)
print(ytest.shape)


(2400, 256, 256, 3)
(2400, 2)
(600, 256, 256, 3)
(600, 2)


In [13]:
from keras.models import Sequential

model = Sequential()

model.add(Conv2D(32,(3,3), activation = 'relu' , input_shape = (256,256,3)))
model.add(MaxPooling2D((2,2)))

model.add(Conv2D(32,(3,3), activation = 'relu'))
model.add(MaxPooling2D((2,2)))

model.add(Flatten())
model.add(Dense(64, activation = 'relu'))
model.add(Dense(NoOfOutputLayer, activation = 'sigmoid'))

model.compile(loss = 'binary_crossentropy', optimizer = 'adam' , metrics = ['accuracy'])

In [18]:
# model.fit(xtrain,ytrain,epochs=5,batch_size=32)
history = model.fit(xtrain, ytrain, batch_size=16, epochs=5)

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

MemoryError: Unable to allocate 1.76 GiB for an array with shape (2400, 256, 256, 3) and data type float32

In [22]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 254, 254, 32)      896       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 127, 127, 32)     0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 125, 125, 32)      9248      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 62, 62, 32)       0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 123008)            0         
                                                                 
 dense_2 (Dense)             (None, 64)               

## save the model

In [23]:
model.save('brain_tumor_model.h5')

## Testing input images

In [24]:
import tensorflow as tf
from keras.utils import load_img
from keras.models import load_model
from keras.utils import img_to_array
from keras.applications.vgg16 import preprocess_input
import numpy as np
import cv2

### load the saved model

In [25]:
model = load_model('brain_tumor_model.h5')

In [26]:
# for check the image |Testng

def check(res):
        p1 = classes
        path = p1
        pred = model.predict(res)
        res = np.argmax(pred)
        res = path[res]
        return (res)


def convert_img_to_tensor2(fpath):
    img = cv2.imread(fpath)
    img = cv2.resize(img, (256, 256))
    res = img_to_array(img)
    res = np.array(res, dtype=np.float16) / 255.0
    res = res.reshape(-1, 256, 256, 3)
    res = res.reshape(1, 256, 256, 3)
    return res

In [28]:
fpath="data/brain-tumor-datasets/yes/y100.jpg"

res = convert_img_to_tensor2(fpath)
msg=check(res)
msg

1/1 [==============================] - 0s 52ms/step


'yes'

In [1]:
loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', loss)
print('Test accuracy:', accuracy)

NameError: name 'model' is not defined